In [1]:
from lionagi.core import System, Instruction
from lionagi.core.execute.structure_executor import StructureExecutor
from lionagi.core.execute.instruction_map_executor import InstructionMapExecutor
from lionagi.core.agent.base_agent import BaseAgent

from lionagi.core.generic.condition import Condition
from lionagi.core import func_to_tool
from lionagi.core.generic.action import ActionSelection

# Math Problem: Optimizing Spending on Notebooks and Stationery

Description:
- The task begins with an inital balance, which is to be used primarily for purchasing notebooks. Each notebook costs $8, and the objective is to buy as many notebooks as possible without exceeding the budget.

- After purchasing the notebooks, with any leftover balance, we can decided whether or not to spend it on less expensive items. The available choices are pen, $5, and pencil, $2. If we choose to buy additional items, we will aim to maximize the quantity without going over the budget.

## Prepare Tools and Actions

In [2]:
async def multiply(number1: float, number2: float):
    """
    multiplication.

    Args:
        number1: First number to multiply.
        number2: Second number to multiply.

    Returns:
        The product of number1, number2

    """
    return number1 * number2

async def divide(number1: float, number2: float):
    """
    division.

    Args:
        number1: dividend.
        number2: divisor.

    Returns:
        The quotient of number1/number2

    """
    return number1 / number2


async def add(number1: float, number2: float):
    """
    addition

    Args:
        number1: First number to add.
        number2: Second number to add.

    Returns:
        The sum of number1, number2
    """
    return number1 + number2

async def minus(number1: float, number2: float):
    """
    subtraction.


    Args:
        number1: minuend.
        number2: subtrahend.

    Returns:
        The difference of number1, number2
    """
    return number1 - number2

In [3]:
calc_tools = func_to_tool([multiply, divide, add, minus])

In [4]:
action = ActionSelection(
    action="followup", action_kwargs={"max_followup":3, "temperature":0.3})

# Start Structure

In [5]:
shopping = StructureExecutor()

system = System("You are a financial management assistant tasked with optimizing spending based on the available balance.")
shopping.add_node(system)

shopping.add_node(calc_tools[0]) # multiply
shopping.add_node(calc_tools[1]) # divide
shopping.add_node(calc_tools[2]) # add
shopping.add_node(calc_tools[3]) # minus

shopping.add_node(action)

## Primary Objective

In [6]:
# primary goal: buy notebooks
instruction1 = Instruction({"primary goal": "You intend primarily for purchasing notebooks as many as possible without exceeding your budget",
                            "notebook price": "$8",
                            "question": "How many books can you purchase and what is the total cost"})

instruction2 = Instruction({"calculate remaining balance": "return the availble balance after buying, please only return a number"})

We use `followup` for both steps

We provide multiplication and division tools to calculate the total cost, and addition and subtraction tools to calculate the balance

In [7]:
# link instructions
shopping.relate_nodes(system, instruction1)
shopping.relate_nodes(instruction1, instruction2)

# link tools
shopping.relate_nodes(instruction1, calc_tools[0])
shopping.relate_nodes(instruction1, calc_tools[1])
shopping.relate_nodes(instruction2, calc_tools[2])
shopping.relate_nodes(instruction2, calc_tools[3])

# link action
shopping.relate_nodes(instruction1, action)
shopping.relate_nodes(instruction2, action)

# Continue Purchasing Options

In [8]:
# only consider to buy addition items when the remaining balance >= item price

class AffordableCondition(Condition):
    def __init__(self, price):
        super().__init__(source_type="executable")
        self.price = price

    def get_remaining_balance(self, branch):
        remaining_balance = branch.execution_responses[-1]
        return int(remaining_balance)

    # executable: BranchExecutor
    def __call__(self, executable):
        balance = self.get_remaining_balance(executable)
        return balance >= self.price

pencil = AffordableCondition(price=2)
pen = AffordableCondition(price=5)

In [9]:
instruction3_pencil = Instruction({"availble choice": "spend your remaining balance if possible without exceeding your budget",
                             "pencil price": "$2",
                             "question": "How many pencils can you purchase and what is the total cost"})

instruction3_pen = Instruction({"availble choice": "spend your remaining balance if possible without exceeding your budget",
                             "pen price": "$5",
                             "question": "How many pencils can you purchase and what is the total cost"})

instruction4 = Instruction({"calculate remaining balance": "return the availble balance after buying"})

In [10]:
shopping.add_node(instruction3_pencil)
shopping.add_node(instruction3_pen)
shopping.add_node(instruction4)

In [11]:
# link intructions
shopping.relate_nodes(instruction2, instruction3_pencil, condition=pencil)
shopping.relate_nodes(instruction2, instruction3_pen, condition=pen)
shopping.relate_nodes(instruction3_pencil, instruction4)
shopping.relate_nodes(instruction3_pen, instruction4)

# link tools
shopping.relate_nodes(instruction3_pencil, calc_tools[0])
shopping.relate_nodes(instruction3_pencil, calc_tools[1])
shopping.relate_nodes(instruction3_pen, calc_tools[0])
shopping.relate_nodes(instruction3_pen, calc_tools[1])
shopping.relate_nodes(instruction4, calc_tools[2])
shopping.relate_nodes(instruction4, calc_tools[3])

# link action
shopping.relate_nodes(instruction3_pencil, action)
shopping.relate_nodes(instruction3_pen, action)
shopping.relate_nodes(instruction4, action)

# Summary

In [12]:
instruction5 = Instruction({"summary": "Provide a summry of items you bought with the remaining balance"})

In [13]:
shopping.add_node(instruction5)

shopping.relate_nodes(instruction2, instruction5) # stop shopping after purchasing notebooks
shopping.relate_nodes(instruction4, instruction5)

# Execute

In [14]:
## output_parser parameter: agent self
## output_parser return: the expected result that should be returned after execution
def output_parser(agent):
    ## return list of all purchasing combinations
    output = []
    for branch in agent.executable.branches.values():
        output.append(branch.execution_responses[-1])
    return output

executable = InstructionMapExecutor()

In [15]:
assistant = BaseAgent(structure=shopping, 
                      executable=executable,
                      output_parser=output_parser)

In [16]:
result = await assistant.execute(context={"balance": 20})

------------------Welcome: system--------------------


system: You are a financial management assistant tasked with optimizing spending based on the available balance.

user: {"primary goal": "You intend primarily for purchasing notebooks as many as possible without exceeding your budget", "notebook price": "$8", "question": "How many books can you purchase and what is the total cost"}

assistant: You can purchase 2 notebooks with a total cost of $16, which is within your budget of $20.

-----------------------------------------------------


...

assistant: 0

-----------------------------------------------------


user: {"summary": "Provide a summry of items you bought with the remaining balance"}

assistant: With your initial balance, you purchased 2 notebooks for a total of $16, leaving you with $4. You then used this remaining balance to buy 2 pencils for $4, fully utilizing your funds.

-----------------------------------------------------


In [17]:
result[0]

'With your initial balance, you purchased 2 notebooks for a total of $16, leaving you with $4. You then used this remaining balance to buy 2 pencils for $4, fully utilizing your funds.'

In [18]:
result[1]

'You purchased 2 notebooks for a total of $16. The remaining balance is $4.'

# Store Structure

### Excel

In [19]:
shopping.to_excel("shopping")

### Neo4j

In [20]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "12345678"
database = "neo4j"

from lionagi.integrations.storage.neo4j import Neo4j
neo4j = Neo4j(uri, user, password, database)

In [22]:
await neo4j.store(shopping, structure_name='shopping')